# Exercice 3

---

Le but de ce notebook est d'étudier le jeu de données de la valeur des crypto afin de pouvoir faire une prédiction des gains possibles qu'on pourrait avoir.

Pour des raisons de taille, le jeu de données n'a pas pu être uploadé avec le github. Il faudra donc le récupérer à [l'adresse suivante](https://www.kaggle.com/c/g-research-crypto-forecasting/data). Seul les jeux de données train.csv et asset_details.csv nous intéressent.

# Import des bibliothèques

In [ ]:
from datetime import datetime
import dateutil.tz
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose

# Chargement des tables dans l'espace de travail

In [ ]:
crypto_train = pd.read_csv("g-research-crypto-forecasting/train.csv")
# supplemental_train est une donnée d'entrainement fournie pour indication. Ne pas utiliser.
#crypto_additional_train = pd.read_csv("g-research-crypto-forecasting/supplemental_train.csv")
crypto_info = pd.read_csv("g-research-crypto-forecasting/asset_details.csv")

In [ ]:
# Piège : la colonne timestamp possède une valeur assez peu lisible.
crypto_train.head(5)

In [ ]:
crypto_info.head()

# Partie 1 - Qualité des données

1) Trouvez le nombre de lignes et de colonnes.
1) Faites un join avec la table des infos pour restituer à chaque cryptomonnaie son nom.
1) Trouvez le nombre de valeur manquantes. Quelle est la crypto-monnaie avec le plus de valeurs manquantes ?
1) Créer une colonne date qui change la colonne timestamp en une vraie date.
1) Timestamp maximal, minimal et nombre de points pour chacun de nos assets. Quelle semble être la granularité de notre timeseries ?
1) Pour chacune des crypto-monnaies, de combien de points disposons-nous ? Que pouvons-nous dire en comparant avec la date minimal pour chacune de nos cryptomonnaies ?

In [ ]:
# Question 1)
print(f"Nous possédons {len(crypto_train)} lignes de données et {len(crypto_train.columns)} colonnes. ")

In [ ]:
crypto_train.info()

In [ ]:
# Question 2) On fusionne les deux tables. Dans notre cas, on fait un left join sur la colonne commune.
crypto_with_name = pd.merge(crypto_train, crypto_info, on="Asset_ID", how="left")

In [ ]:
# Question 3) trouvons le nombre de valeurs manquantes.

# Il est déconseillé d'utiliser la commande .describe() car nous allons alors effectuer des calculs de moyennes et autres variables.
# Dans notre cas, notre dataset est trop grand pour calculer tout ça.
# crypto_train.describe()

# A la place, nous allons nous borner à faire un calcul simple. Par colonne, nous disposons du nombre suivant de valeurs nulles.
crypto_with_name.isna().sum()

In [ ]:
# On constate que la majorité des valeurs nulles proviennent de la colonne target. Dans le cas des Timestamp, avoir des valeurs nulles est très grave.
# En effet, on coupe la continuité du temps et comme le rapport de chaque variable avec les précédentes est important, on se prive d'une information capitale.

# Nous séparons le nombre de valeurs nulles par crypto
crypto_with_name.loc[crypto_with_name["Target"].isna()].groupby("Asset_Name").count()["timestamp"].sort_values(ascending=False)
# Le fait que les monnaies les plus connues soient les plus documentées nous semble pertinent.

In [ ]:
# Question 4) On utilise la librairie datetime qui possède une bibliothèque associée.
tz_keep = dateutil.tz.tzutc()
crypto_with_name["datetime"] = crypto_with_name["timestamp"].apply(lambda x : datetime.fromtimestamp(x, tz=tz_keep))

In [ ]:
# Question 5) On peut appliquer les méthodes min et max aux dates ==> Pas de soucis.
print(f"Les extremums des dates sont {crypto_with_name['datetime'].min()} pour le minimum et {crypto_with_name['datetime'].max()} pour le maximum.") 

In [ ]:
# En regardant les valeurs classées, on devine que la granularité des données est une minute.
sorted(crypto_with_name['datetime'].unique())

In [ ]:
# Question 6) On compte le nombre de points de données que l'on possède pour chacune de nos cryptomonnaies.
crypto_with_name.groupby('Asset_Name')['timestamp'].count().sort_values(ascending=False)

In [ ]:
# On possède beaucoup de valeurs depuis le premier janvier 2018 MAIS pour ces monnaies, on ne possède pas le même nombre de points.
# Il doit manquer des points intermédiaires ==> Ouch
crypto_with_name.groupby('Asset_Name')['datetime'].min().sort_values(ascending=False)

In [ ]:
# On possède toutes les monnaies jusqu'au 21 septembre 2021. On va effectivement avoir des valeurs manquantes au milieu.
crypto_with_name.groupby('Asset_Name')['datetime'].max().sort_values(ascending=False)

In [ ]:
crypto_with_name.columns

# Partie 2 - Analyse données

---

## Explication des valeurs des colonnes.

- **timestamp** : La minute considérée par la colonne.
- **Asset_ID** & **Asset_Name** : identifiant de la crypto-monnaie.
- **Count** : Nombre d'échanges de la monnaie lors de la minute.
- **Open** & **Close** : Prix d'ouverture et de fermeture (prix au début et à la fin de la minute).
- **High** & **Low** : Prix maximal et minimal atteint dans la minute.
- **Volume** : Montant total échangé lors de la minute.
- **VWAP** : Prix moyen sur la minute pondéré par volume de vente.
- **Target** : Log du retour sur investissement sur les 15 prochaines minutes auquel on a appliqué une transformation. f(log(prix à l'instant t+16) - log(prix à l'instant t+1))
- **Weight** : Lié à la cryptomonnaie. Importance de la crypto-monnaie (valeur arbitraire).

---

Nous n'allons pas utiliser les valeurs de weight et target. Nous allons essayer de prédire la VWAP de la monnaie avec 7j d'avance.

---

1) Calculez la corrélations entre les valeurs moyennes des différentes monnaies sur l'année 2021. Ce résultat vous étonne-t-il ?
1) Au vue de la question précédente, nous allons nous intéresser uniquement au bitcoin pour l'instant. Identifiez les timestamp manquantes pour le bitcoin.
1) Quand sont atteints les optimums et pourquoi ? Quand sont atteint les plus grands écarts min max en valeur puis en %
1) Quels jours de la semaine, quels mois de l'année et quelles semaines de l'années y a-t-il le plus de transactions (en moyenne) ? Décomposez les statistiques selon les périodes qui vous semblent pertinentes.
1) Faire un graphe qui présente par semaine le prix moyen des cryptomonnaies avec leur min et le max (dans le même graphe).
1) Autocorrelation décalée pour voir s'il y a des périodicités.
1) Faire des graphe qui présentent la tendance et  la saisonnalité des données.
1) Quelle méthode utiliseriez-vous pour faire jeux de test et train ?

In [ ]:
# Question 1) On commence par isoler l'année 2021
crypto_2021 = crypto_with_name.loc[crypto_with_name["datetime"].dt.year == 2021]
# Afin de faire pivoter la ligne des Asset_Name et la transformer en colonne, on utilise la commande pivot
pivoted_table = crypto_2021.pivot(index='datetime', columns='Asset_Name', values='VWAP')

In [ ]:
# On constate que le coefficient de corrélation entre les différentes monnaies est assez fort.
# Plusieurs monnaies étant liées, cela semble logique.

matrix = pivoted_table.corr().round(2)
sns.heatmap(matrix, annot=True, vmax=1, vmin=0, center=0, cmap='vlag')

In [ ]:
# Question 2) Afin de trouver s'il manque des minutes sur notre timestamp, nous allons construire un dataset qui contient toutes les position (minute par minute) et le comparer à notre table crypto.

crypto_bitcoin = crypto_with_name.loc[crypto_with_name["Asset_Name"] == "Bitcoin"].reset_index(drop=True)
date_range_df = pd.date_range(crypto_bitcoin["datetime"].min(), crypto_bitcoin["datetime"].max(), freq="min")
datetime_column = pd.DataFrame(date_range_df).rename({0: "datetime"}, axis=1)
print(f"Il manque {len(date_range_df) - len(crypto_bitcoin)} minutes dans notre dataframe.")

In [ ]:
# Afin de trouver les minutes manquantes, nous procédons à une comparaison
missing_values = pd.concat([datetime_column, crypto_bitcoin[["datetime"]]]).drop_duplicates(keep=False)
missing_values

In [ ]:
# On constate que tous les mois, on perd une minute (minuit du premier jour de chaque mois). Vérification ci-dessous pour le mois de février 2018
crypto_bitcoin.loc[(crypto_bitcoin["datetime"] > "2018-01-31 23:57:00+00:00") & (crypto_bitcoin["datetime"] < "2018-02-01 00:02:00+00:00")]

In [ ]:
# On constante aussi que le 2019-10-16, il manque des minutes et pareil à d'autres moments.
# Que faire de ces données vides ? Dans le cas d'une minute par ci par là, on peut mettre des moyennes. Un jour complet, c'est plus périlleux.
# Note : cas des données de ventes : si un jour manque c'est sans doute parce qu'on n'a pas eu de ventes (on met donc ventes = 0). Dans notre cas, c'est plus compliqué.
missing_values.loc[(missing_values["datetime"] > "2019-10-16 00:00:00+00:00") & (missing_values["datetime"] < "2019-10-17 00:00:00+00:00")]

In [ ]:
# Question 3) Question ambigüe. Maximum en moyenne ? Maximum de rendement ? Maximum à ouverture ?
# Dans notre cas, on regarde la moyenne.
print(f"La valeur moyenne maximale sur une minute est atteinte le {crypto_bitcoin.loc[crypto_bitcoin['VWAP'] == crypto_bitcoin['VWAP'].max()]['datetime'].iloc[0]}.")
print(f"La valeur moyenne minimale sur une minute est atteinte le {crypto_bitcoin.loc[crypto_bitcoin['VWAP'] == crypto_bitcoin['VWAP'].min()]['datetime'].iloc[0]}.")
# Nous savons que la valeur du bitcoin a globalement augmenté avant la crise de 2022.

In [ ]:
crypto_bitcoin.columns

In [ ]:
crypto_bitcoin["ecart_min_max"] = crypto_bitcoin["High"] - crypto_bitcoin["Low"]
# L'écart en pourcentage doit être calculé en divisant par le min, le max ou la moyenne ? Dans notre cas, le moyenne semble le plus sensé mais débattable
crypto_bitcoin["ecart_min_max_pourcent"] = (crypto_bitcoin["High"] - crypto_bitcoin["Low"]) / crypto_bitcoin["VWAP"] * 100

print(f"Le plus grand écart entre minimum et maximum est atteint le {crypto_bitcoin.loc[crypto_bitcoin['ecart_min_max'] == crypto_bitcoin['ecart_min_max'].max()]['datetime'].iloc[0]}.")
print(f"Le plus grand écart en pourcentage entre minimum et maximum est atteint le {crypto_bitcoin.loc[crypto_bitcoin['ecart_min_max_pourcent'] == crypto_bitcoin['ecart_min_max_pourcent'].max()]['datetime'].iloc[0]}.")

En terme d'écart, on constate que le maximum en écart valeur est atteint pas très loin du jour où on atteint le maximum dans l'absolu. Cela vient du faire que plus la valeur augmente, plus (logiquement) les écarts augmentent. Cette statistique n'est donc pas très utile. L'écart en pourcentage nous aide plus et nous permet de repérer des périodes d'instabilité dans le bitcoin.

In [ ]:
# Question 4) Nous groupons nos données selon les différentes granularités.
crypto_bitcoin["year"] = crypto_bitcoin["datetime"].dt.year
crypto_bitcoin["month"] = crypto_bitcoin["datetime"].dt.month
crypto_bitcoin["week_number"] = crypto_bitcoin["datetime"].apply(lambda x : x.isocalendar().week)
crypto_bitcoin["week_day"] = crypto_bitcoin["datetime"].dt.weekday # Jour entre 0 et 6

In [ ]:
# Que les cryptos augmentent en prix avec le nombre d'échange semble cohérent.
crypto_bitcoin.groupby("year")["Volume"].mean()

In [ ]:
# Les mois les plus populaires sont mars, mai et Novembre...
crypto_bitcoin.groupby("month")["Volume"].mean().sort_values(ascending=False)

In [ ]:
# On constate que la domination de mars vient surtout de mars 2020 qui a été très fort.
crypto_bitcoin.groupby(["year", "month"])["Volume"].mean().sort_values(ascending=False)

In [ ]:
# En regardant la médiane, on se rend compte d'une réalité un peu différente (et amusante) : le trading a eu lieu en début d'année.
crypto_bitcoin.groupby("month")["Volume"].median().sort_values(ascending=False)

In [ ]:
# Les semaines de l'année ne semblent pas révéler d'informations qu'on ne possède pas.
crypto_bitcoin.groupby("week_number")["Volume"].mean().sort_values(ascending=False)

In [ ]:
# Les jours de la semaines sont par contre très intéressants. On constate que les weekends sont derniers : les traders ne travaillent pas.
crypto_bitcoin.groupby("week_day")["Volume"].mean().sort_values(ascending=False)

In [ ]:
# Nous voulons étudier les valeurs moyennes. Gors piège : la moyenne des moyennes n'est pas la moyenne. Il faut prendre en compte les volumes.

crypto_bitcoin["year_week"] = crypto_bitcoin["year"].apply(str) + " - " + crypto_bitcoin["week_number"].apply(lambda x : str(x) if len(str(x)) == 2 else '0' + str(x))
crypto_bitcoin["weighted_mean"] = crypto_bitcoin["VWAP"] * crypto_bitcoin["Volume"]
mean_graph = (crypto_bitcoin.groupby(["year_week"])["weighted_mean"].sum() / crypto_bitcoin.groupby(["year_week"])["Volume"].sum()).reset_index().rename({0:"weighted_mean"}, axis=1)
max_graph = crypto_bitcoin.groupby(["year_week"])["High"].max().reset_index()
min_graph = crypto_bitcoin.groupby(["year_week"])["Low"].min().reset_index()

In [ ]:
# Question 5
# Il est possible de faire plusieurs plots (en fonction de ce que l'on souhaite présenter).
# On décrypte la demande : prix ==> Moyenne des moyennes; max ==> max des max; min ==> min des min

fig = go.Figure()
fig.add_trace(go.Scatter(x=mean_graph["year_week"], y=mean_graph["weighted_mean"], name=f"Prix moyen hebdomadaire", mode="markers+lines"))
fig.add_trace(go.Scatter(x=max_graph["year_week"], y=max_graph["High"], name=f"Prix maximum hebdomadaire", mode="lines"))
fig.add_trace(go.Scatter(x=min_graph["year_week"], y=min_graph["Low"], name=f"Prix minimum hebdomadaire", mode="lines"))
fig.update_layout(title=f"Prix moyen, min et max hebdomadaire du bitcoin", xaxis_title=f"Semaine de l'année", yaxis_title="Prix du bitcoin")
fig.show()

In [ ]:
# Pour l'instant, on ne s'est pas intéressé au target. Voyons voir s'il a quelque chose à nous dire
# On possède une donnée centrée (rappelez-vous, on a un log). La moyenne sur toute période de temps un tant soit peu grande sera égale à 0.
# Qu'est-ce qu'une période grande ? Une journée = 60 * 24 points. C'est grand.
test_graph = crypto_bitcoin.groupby(["year_week"])["Target"].mean().reset_index()
fig = go.Figure()
fig.add_trace(go.Scatter(x=test_graph["year_week"], y=test_graph["Target"], name=f"Target", mode="lines"))
fig.update_layout(title=f"Evolution du target pour le bitcoin", xaxis_title=f"Semaine de l'année", yaxis_title="target")
fig.show()

In [ ]:
# Afin de pouvoir regarder avec un peu de clarté et sans avoir à plot 1 millions de points de données, on va se focaliser sur 2020.
graph_2020 = crypto_bitcoin.loc[crypto_bitcoin["year"] == 2020]
fig = go.Figure()
fig.add_trace(go.Scatter(x=graph_2020["datetime"], y=graph_2020["Target"], name=f"Target", mode="lines"))
fig.update_layout(title=f"Evolution du target pour le bitcoin sur l'année 2020", xaxis_title=f"datetime", yaxis_title="target")
fig.show()

# On constate qu'on possède beaucoup de "bruits". Avoir la valeur absolue des précédents targets parait intéressant.
# On constate que les valeurs sont faibles ==> Faire du min max pour ramener entre 0 et 1 parait bien.

In [ ]:
# Allons plus loin. Vous êtes data scientist, vous avez la demande et vous vous dites "on peut faire mieux".
# Il existe un graphe appelé Candle sticks graph. Il nous faut le max, prix d'ouverture et de fermeture pour l'utiliser.
# Le prix d'ouverture de la semaine est le prix d'ouverture du premier jour, le prix de fermeture est le prix de fermeture du dernier jour.

opening_graph = crypto_bitcoin.loc[(crypto_bitcoin["week_day"] == 0) & (crypto_bitcoin["datetime"].dt.time.apply(str) == "00:01:00")][["year_week", "Open"]]
closing_graph = crypto_bitcoin.loc[(crypto_bitcoin["week_day"] == 6) & (crypto_bitcoin["datetime"].dt.time.apply(str) == "23:59:00")][["year_week", "Close"]]

fig = go.Figure(data=[go.Candlestick(x=max_graph['year_week'],
                open=opening_graph['Open'],
                high=max_graph['High'],
                low=min_graph['Low'],
                close=closing_graph['Close'])])
fig.update_layout(title=f"Prix d'ouverture, min et max hebdomadaire du bitcoin", xaxis_title=f"Semaine de l'année", yaxis_title="Prix du bitcoin")

fig.show()

In [ ]:
# Question 6)
# Jettons un coup d'oeil à l'autocorrélation... On peut regarder pour les minutes, jours, semaines, mois, années...
sm.tsa.acf(crypto_bitcoin.groupby(["year", "week_number"])["VWAP"].mean().values, nlags=52)

In [ ]:
# Question 7)
# Qu'est-ce que la tendance, la saisonnalité et le bruit ?
# Tendance = moyenne mobile sur notre période de référence.
# Saisonnalité = moyenne de l'écart à la tendance pendant notre saison.
# Bruit = Différence entre valeur réelle et tendance + saisonnalité.
# On utilise la fonction seasonal_decompose.

# Attention, on choisit two_sided=True, on utilise des données de l'avenir ==> On ne peut pas utiliser notre valeur comme feature.
seasonal_data = crypto_bitcoin.set_index("datetime")
sd = seasonal_decompose(seasonal_data["VWAP"], period=24*60*7*4*2, two_sided=True) # La période proposée est de 2 mois, soit 24*60*7*4*2 minutes (points)

In [ ]:
# On récupère la tendance et on la trace par rapport aux valeurs réelles.
seasonal_data["observed"] = sd.observed
seasonal_data["trend"] = sd.trend
mean_observed = seasonal_data.reset_index().groupby("year_week")[["observed", "trend"]].mean().reset_index()

# Attention, la période choisie pour faire notre tendance doit dépendre de notre problème : dans notre cas avec de grandes variations, 2 mois semble adapté.
# Si notre but était de faire du trading au jour le jour, il aurait fallu choisir une période plus réduite sur laquelle lisser.
fig = go.Figure()
fig.add_trace(go.Scatter(x=mean_observed["year_week"], y=mean_observed["observed"], name=f"Prix moyen hebdomadaire", mode="markers+lines"))
fig.add_trace(go.Scatter(x=mean_observed["year_week"], y=mean_observed["trend"], name=f"Tendance globale hebdomadaire", mode="lines"))
fig.update_layout(title=f"Prix moyen et tendance hebdomadaire du bitcoin", xaxis_title=f"Semaine de l'année", yaxis_title="Prix du bitcoin")
fig.show()

In [ ]:
# On regarde le graphe de saisonnalité dans notre cas et on ne peut pas s'empêcher d'être déçu. Les valeurs sont entre 500 et -500.
# A l'échelle du bitcoin, c'est minuscule. Pas beaucoup d'intérêt.
seasonal_data["seasonal"] = sd.seasonal
mean_observed_seasonal = seasonal_data.reset_index().groupby("year_week")[["seasonal"]].mean().reset_index()
fig = go.Figure()
fig.add_trace(go.Scatter(x=mean_observed_seasonal["year_week"], y=mean_observed_seasonal["seasonal"], name=f"saisonnalité moyenne hebdomadaire", mode="markers+lines"))
fig.update_layout(title=f"Saisonnalité moyenne hebdomadaire du bitcoin", xaxis_title=f"Semaine de l'année", yaxis_title="Valeur du bitcoin")
fig.show()

Question 8 : Plusieurs éléments sont à prendre en compte.
- Le temps est linéaire. Pour constituer nos jeux d'entrainement et de validation, il faut que les timestamp des données d'entrainement soient inférieurs à celui des jeux de validation. Typiquement dans notre cas : on prend tous le temps avant mi-2021 comme entrainement et le reste comme jeu de validation.
- Les différences échelles d'une monnaie à l'autre sont très importantes. Il faut faire une transformatin. Notre Target semble normalisé mais dans les cas où il faut deviner un prix, on aurait plutôt cherché à prédire un pourcentage de croissance ou un truc comme ça qui exclue les échelles.
- Il faut des échantillons de toutes les monnaies. Si ce n'est pas possible, on ne peut pas faire de prédictions.
- Dans le cas de données périodiques, il faut prendre plusieurs périodes. Au moins 2.

Partie 3 - Prédiction

---

Notre but est de prédire la VWAP avec une semaine d'avance.
Pour se simplifier la tâche, nous allons agglomérer les données à la maille jour avec le max qui est le max de la journée, le min est le min, le volume est la somme des volumes, la VWAP qui est la VWAP pondérée...

---

1) Mettez en forme le jeu de données et créez la variable cible à prédire.
1) Rajoutez la semaine de l'année en tant que variable catégorique.
1) Afin de préparer les prédictions, normalisez les données.
1) Ajoutez comme features un historique des valeurs sur les dernières jours et toutes autres valeurs qui semblent intéressantes. (Dans les problèmes à saisonnalité, on aurait récupéré les valeurs des précédentes saisons)
1) En choisissant judicieusement vos jeux d'entrainement et de tests, faites des prédictions en utilisant une régression linéaire. Evaluez les résultats.
1) Transformez la variable cible, au lieu de prédire la valeur à jour + 1, nous allons prédire la différence entre valeur actuelle et valeur à jour + 1. Comparez les résultats, conclure.
- SARIMAX (ne fonctionne que pour les données stationnaires + ne prend pas en compte les différentes variables) ==> Inutile ici. Notre problème n'est pas stationnaire.
- Modèles "classiques". Problème : on ne prédit pas très bien le rapport entre les features. 

In [ ]:
# Question 1)

# On commence par agréger les données à la maille jour.
crypto_bitcoin["date"] = crypto_bitcoin["datetime"].dt.date

opening_per_day = crypto_bitcoin.loc[(crypto_bitcoin["datetime"].dt.time.apply(str) == "00:01:00")][["date", "Open"]].set_index("date")
closing_per_day = crypto_bitcoin.loc[(crypto_bitcoin["datetime"].dt.time.apply(str) == "23:59:00")][["date", "Close"]].set_index("date")
bitcoin_per_day = crypto_bitcoin[["date", "month", "week_number", "week_day"]].drop_duplicates(["date", "month", "week_number", "week_day"]).set_index("date")

df_group = crypto_bitcoin.groupby("date")
high_col = df_group["High"].apply(max)
low_col = df_group["Low"].apply(min)
vol_col = df_group[["Count", "Volume", "weighted_mean"]].apply(sum)
vol_col["weighted_mean"] = vol_col["weighted_mean"] / vol_col["Volume"]

In [ ]:
# On constitue notre base de données
# Utiliser l'année comme variable d'apprentissage n'est pas pertinent. On va changer d'année. Les mois et semaines de l'années nous semblent pertinentd.

bitcoin_per_day["High"] = high_col
bitcoin_per_day["Low"] = low_col
bitcoin_per_day["Open"] = opening_per_day
bitcoin_per_day["Close"] = closing_per_day
bitcoin_per_day[["Count", "Volume", "weighted_mean"]] = vol_col
bitcoin_per_day["ecart_min_max"] = bitcoin_per_day["High"] - bitcoin_per_day["Low"]
bitcoin_per_day["ecart_min_max_pourcent"] = bitcoin_per_day["ecart_min_max"] / bitcoin_per_day["weighted_mean"]
bitcoin_per_day = bitcoin_per_day.reset_index()

In [ ]:
# On vérifie s'il nous manque des jours.
# Dans notre cas, il ne nous manque pas de jours.
len(pd.date_range(bitcoin_per_day["date"].min(), bitcoin_per_day["date"].max())) == len(bitcoin_per_day)

In [ ]:
# Afin de construire notre variable cible, on applique la méthode shift qui décale nos valeurs d'un jour.
bitcoin_per_day = bitcoin_per_day.set_index("date")
bitcoin_per_day["target"] = bitcoin_per_day["weighted_mean"].shift(-7)
bitcoin_per_day["real_target"] = bitcoin_per_day["target"]
bitcoin_per_day = bitcoin_per_day.reset_index()

In [ ]:
# Question 2 & 3) Est-ce qu'il faut normaliser les données en utilisant uniquement les valeurs du passé ou en utilisant toutes les valeurs ?
# Dans les cas où on possède des données avec une distribution stable, il vaut mieux utiliser toutes les valeurs. Nos données sont-elles stables ? A court terme oui. Avec plusieurs points d'inflexion.
# On va par contre appliquer un log à nos valeurs afin de les ramener dans un écart raisonnable.

col_to_log = ['High', 'Low', 'Open', 'Close', 'Count', 'Volume', 'weighted_mean', 'target']
col_to_encode = ['month', 'week_number', 'week_day']

# On applique un logarithme
for my_col in col_to_log:
    bitcoin_per_day[my_col] = bitcoin_per_day[my_col].apply(lambda x : np.log10(x+1))

# On fait du target encoding
for my_col in col_to_encode:
    inter_groupby = bitcoin_per_day.groupby(my_col)
    inter_count_target = inter_groupby[["target"]].sum()
    inter_count_target["target"] = inter_count_target["target"] / inter_groupby["target"].count()
    inter_count_target = inter_count_target.reset_index().rename({"target" : my_col + "_te"}, axis = 1)
    bitcoin_per_day = pd.merge(bitcoin_per_day, inter_count_target, on=my_col)

# On normalise
# On conserve la moyenne et l'ecart type de notre variable cible.
for my_col in bitcoin_per_day.columns:
    if my_col not in col_to_encode + ["date", "target", "real_target"]:
        bitcoin_per_day[my_col] = (bitcoin_per_day[my_col] - bitcoin_per_day[my_col].mean()) / bitcoin_per_day[my_col].std()
    if my_col == "target":
        target_mean = bitcoin_per_day[my_col].mean()
        target_std = bitcoin_per_day[my_col].std()
        bitcoin_per_day[my_col] = (bitcoin_per_day[my_col] - bitcoin_per_day[my_col].mean()) / bitcoin_per_day[my_col].std()


In [ ]:
# Question 4)
bitcoin_per_day = bitcoin_per_day.sort_values("date").set_index("date")
bitcoin_per_day["last_day_value"] = bitcoin_per_day["weighted_mean"].shift(1)
bitcoin_per_day["last_day_growth"] = bitcoin_per_day["weighted_mean"] - bitcoin_per_day["last_day_value"]
bitcoin_per_day["last_week_value"] = bitcoin_per_day["weighted_mean"].shift(7)
bitcoin_per_day["last_week_growth"] = bitcoin_per_day["weighted_mean"] - bitcoin_per_day["last_week_value"]

In [ ]:
# On remplit les valeurs nulles qu'on a crée de cette manière
bitcoin_per_day = bitcoin_per_day.bfill()
bitcoin_per_day[["Open", "Close"]] = bitcoin_per_day[["Open", "Close"]].ffill()

In [ ]:
# Question 5)
np.random.seed(42)

bitcoin_ml = bitcoin_per_day.dropna()

# Note : on possède des variables corrélées. On aurait sans doute du appliquer de la ridge regression ou de la lasso regression.
# On aurait ainsi pu éliminer les variables inutiles.
col_ml = ['High', 'Low', 'Open', 'Close',
       'Count', 'Volume', 'weighted_mean', 'ecart_min_max',
       'ecart_min_max_pourcent', 'month_te', 'week_number_te',
       'week_day_te', 'last_day_value', 'last_day_growth', 'last_week_value',
       'last_week_growth']

# On récupère les valeurs de nos jeux d'entrainement et de test
train_set = bitcoin_ml.head(4 * len(bitcoin_ml)//5)
test_set = bitcoin_ml.tail(len(bitcoin_ml) - len(train_set))
x_train = train_set[col_ml]
y_train = train_set["target"]
x_test = test_set[col_ml]
y_test = test_set[["real_target"]].reset_index(drop=True)

# On crée notre modèle de ML
my_reg = LinearRegression().fit(x_train, y_train)

In [ ]:
# On observe les performances de notre modèle.
# Afin de voir s'il s'en sort bien, on le compare à l'erreur qu'on aurait eu si on avait dit "la nouvelle valeur = ancienne valeur + croissance depuis la semaine dernière"
test_prediction = 10**(my_reg.predict(x_test) * target_std + target_mean)
dumb_estimator = pd.DataFrame(10**((test_set["weighted_mean"] + test_set["last_week_growth"])* target_std + target_mean)).reset_index(drop=True).rename({0 : "dumb_estimator"}, axis = 1)
y_test["AbsDifference_ml"] = (y_test["real_target"] - pd.DataFrame(test_prediction, columns = ["real_target"])["real_target"]).apply(abs)
y_test["AbsDifference_estim"] = (y_test["real_target"] - dumb_estimator["dumb_estimator"]).apply(abs)

# Notre modèle de ml fait mieux que notre estimateur mais c'est pas flagrant.
fig = go.Figure()
fig.add_trace(go.Box(x=y_test["AbsDifference_ml"], name="Erreur pour ml"))
fig.add_trace(go.Box(x=y_test["AbsDifference_estim"], name="Erreur pour estimateur"))
fig.update_layout(title=f"Comparaison des valeurs d'erreurs en fonction du modèle de prédiction", xaxis_title=f"Valeur d'erreur")
fig.show()

In [ ]:
# Comparaison visuel du prix du bitcoin vs notre estimateur

fig = go.Figure()
fig.add_trace(go.Scatter(x=test_set.index, y=y_test["real_target"], name=f"Prix réel du bitcoin", mode="markers+lines"))
fig.add_trace(go.Scatter(x=test_set.index, y=pd.DataFrame(test_prediction, columns = ["real_target"])["real_target"], name=f"Prédiction ML", mode="lines"))
fig.add_trace(go.Scatter(x=test_set.index, y=dumb_estimator["dumb_estimator"], name=f"Prédiction estimateur", mode="lines"))
fig.update_layout(title=f"Evolution du prix du bitcoin au cours du temps", xaxis_title=f"Date", yaxis_title="Prix du bitcoin")
fig.show()

In [ ]:
# Question 6) On refait un peu tout depuis la question 1

# On commence par agréger les données à la maille jour.
crypto_bitcoin["date"] = crypto_bitcoin["datetime"].dt.date

opening_per_day = crypto_bitcoin.loc[(crypto_bitcoin["datetime"].dt.time.apply(str) == "00:01:00")][["date", "Open"]].set_index("date")
closing_per_day = crypto_bitcoin.loc[(crypto_bitcoin["datetime"].dt.time.apply(str) == "23:59:00")][["date", "Close"]].set_index("date")
bitcoin_per_day = crypto_bitcoin[["date", "month", "week_number", "week_day"]].drop_duplicates(["date", "month", "week_number", "week_day"]).set_index("date")

df_group = crypto_bitcoin.groupby("date")
high_col = df_group["High"].apply(max)
low_col = df_group["Low"].apply(min)
vol_col = df_group[["Count", "Volume", "weighted_mean"]].apply(sum)
vol_col["weighted_mean"] = vol_col["weighted_mean"] / vol_col["Volume"]

# On constitue notre base de données
# Utiliser l'année comme variable d'apprentissage n'est pas pertinent. On va changer d'année. Les mois et semaines de l'années nous semblent pertinentd.

bitcoin_per_day["High"] = high_col
bitcoin_per_day["Low"] = low_col
bitcoin_per_day["Open"] = opening_per_day
bitcoin_per_day["Close"] = closing_per_day
bitcoin_per_day[["Count", "Volume", "weighted_mean"]] = vol_col
bitcoin_per_day["ecart_min_max"] = bitcoin_per_day["High"] - bitcoin_per_day["Low"]
bitcoin_per_day["ecart_min_max_pourcent"] = bitcoin_per_day["ecart_min_max"] / bitcoin_per_day["weighted_mean"]
bitcoin_per_day = bitcoin_per_day.reset_index()

# Afin de construire notre variable cible, on applique la méthode shift qui décale nos valeurs d'un jour.
bitcoin_per_day = bitcoin_per_day.set_index("date")
bitcoin_per_day["target"] = bitcoin_per_day["weighted_mean"].shift(-7)
bitcoin_per_day["real_target"] = bitcoin_per_day["target"] 
bitcoin_per_day["target"] = (bitcoin_per_day["target"] - vol_col["weighted_mean"]) / vol_col["weighted_mean"]
bitcoin_per_day["real_weighted_mean"] = vol_col["weighted_mean"]
bitcoin_per_day = bitcoin_per_day.reset_index()

# On va par contre appliquer un log à nos valeurs afin de les ramener dans un écart raisonnable.

col_to_log = ['High', 'Low', 'Open', 'Close', 'Count', 'Volume', 'weighted_mean']
col_to_encode = ['month', 'week_number', 'week_day']

# On applique un logarithme
for my_col in col_to_log:
    bitcoin_per_day[my_col] = bitcoin_per_day[my_col].apply(lambda x : np.log10(x+1))

# On fait du target encoding
for my_col in col_to_encode:
    inter_groupby = bitcoin_per_day.groupby(my_col)
    inter_count_target = inter_groupby[["target"]].sum()
    inter_count_target["target"] = inter_count_target["target"] / inter_groupby["target"].count()
    inter_count_target = inter_count_target.reset_index().rename({"target" : my_col + "_te"}, axis = 1)
    bitcoin_per_day = pd.merge(bitcoin_per_day, inter_count_target, on=my_col)

# On normalise
# On conserve la moyenne et l'ecart type de notre variable cible.
for my_col in bitcoin_per_day.columns:
    if my_col not in col_to_encode + ["date", "target", "real_target", "real_weighted_mean"]:
        bitcoin_per_day[my_col] = (bitcoin_per_day[my_col] - bitcoin_per_day[my_col].mean()) / bitcoin_per_day[my_col].std()
    if my_col == "target":
        target_mean = bitcoin_per_day[my_col].mean()
        target_std = bitcoin_per_day[my_col].std()
        bitcoin_per_day[my_col] = (bitcoin_per_day[my_col] - bitcoin_per_day[my_col].mean()) / bitcoin_per_day[my_col].std()

bitcoin_per_day = bitcoin_per_day.sort_values("date").set_index("date")
bitcoin_per_day["last_day_value"] = bitcoin_per_day["weighted_mean"].shift(1)
bitcoin_per_day["last_day_growth"] = (bitcoin_per_day["weighted_mean"] - bitcoin_per_day["last_day_value"]) / bitcoin_per_day["weighted_mean"]
bitcoin_per_day["last_week_value"] = bitcoin_per_day["weighted_mean"].shift(7)
bitcoin_per_day["last_week_growth"] = (bitcoin_per_day["weighted_mean"] - bitcoin_per_day["last_week_value"])/bitcoin_per_day["weighted_mean"]

# On remplit les valeurs nulles qu'on a crée de cette manière
bitcoin_per_day = bitcoin_per_day.bfill()
bitcoin_per_day[["Open", "Close"]] = bitcoin_per_day[["Open", "Close"]].ffill()

np.random.seed(42)

bitcoin_ml = bitcoin_per_day.dropna()

# Note : on possède des variables corrélées. On aurait sans doute du appliquer de la ridge regression ou de la lasso regression.
# On aurait ainsi pu éliminer les variables inutiles.
col_ml = ['High', 'Low', 'Open', 'Close',
       'Count', 'Volume', 'weighted_mean', 'ecart_min_max',
       'ecart_min_max_pourcent', 'month_te', 'week_number_te',
       'week_day_te', 'last_day_value', 'last_day_growth', 'last_week_value',
       'last_week_growth']

# On récupère les valeurs de nos jeux d'entrainement et de test
train_set = bitcoin_ml.head(4 * len(bitcoin_ml)//5)
test_set = bitcoin_ml.tail(len(bitcoin_ml) - len(train_set))
x_train = train_set[col_ml]
y_train = train_set["target"]
x_test = test_set[col_ml]
y_test = test_set[["real_target", "real_weighted_mean"]].reset_index(drop=True)

# On crée notre modèle de ML
my_reg = LinearRegression().fit(x_train, y_train)

In [ ]:
# On observe les performances de notre modèle.
# On constate que l'erreur médiane est inférieure à précédemment (youhou !)
test_prediction = my_reg.predict(x_test) * target_std + target_mean
y_test["AbsDifference_ml"] = (y_test["real_target"] - (test_prediction * y_test["real_weighted_mean"] + y_test["real_weighted_mean"])).apply(abs)

# Notre modèle de ml fait mieux que notre estimateur mais c'est pas flagrant.
fig = go.Figure()
fig.add_trace(go.Box(x=y_test["AbsDifference_ml"], name="Erreur pour ml"))
fig.update_layout(title=f"Comparaison des valeurs d'erreurs en fonction du modèle de prédiction", xaxis_title=f"Valeur d'erreur")
fig.show()

In [ ]:
# Comparaison visuel du prix du bitcoin vs notre estimateur

fig = go.Figure()
fig.add_trace(go.Scatter(x=test_set.index, y=y_test["real_target"], name=f"Prix réel du bitcoin", mode="markers+lines"))
fig.add_trace(go.Scatter(x=test_set.index, y=test_prediction * y_test["real_weighted_mean"] + y_test["real_weighted_mean"], name=f"Prédiction ML", mode="lines"))
fig.update_layout(title=f"Evolution du prix du bitcoin au cours du temps", xaxis_title=f"Date", yaxis_title="Prix du bitcoin")
fig.show()

En regardant les figures, on constate que le deuxième modèle se débrouille mieux que le premier en moyenne (il est plus simple de deviner un pourcentage entre -50 et 50 qu'une valeur potentiellement très élevée de croissance). Néanmoins, on possède un modèle qui "triche" : il prend plus ou moins la même valeur que la veille ! C'est malin mais c'est pas fou.

On demeure cependant insatisfait du résultat.